![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [62]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head(3)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.00,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.46,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.50,u,g,q,h,1.50,t,f,0,g,824,+


In [63]:
# Things to do : preprocess, train, test, improve
"""Adding missing values"""
## Preprocessing: remove or replace the missing values
import numpy as np
cc_apps.replace('?', np.nan, inplace=True)
cc_apps.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.00,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.46,u,g,q,h,3.04,t,t,6,g,560,+


In [64]:
"""The below approach of converting categorical cols and numerical cols separately does not work as it leads to complications in the arrangement of columns and hence we avoid this way..."""
# ## Preprocessing: Imputation
# df_cat = cc_apps.copy()

# df_cat = df_cat.select_dtypes(include='object')
# for i in df_cat.columns:
#     value = df_cat.iloc[i].value_counts()[0] # getting the most frequent value
#     df_cat.iloc[i].replace(np.nan, value, inplace=True)
# # Categorical columns
# df_cat = pd.get_dummies(df_cat, drop_first=True)
# df_cat.head()

# df_num = cc_apps.copy()

# df_num = df_num.select_dtypes(exclude='object')
# for i in df_num.columns:
#     value = df_num.iloc[i].mean() # getting the most frequent value
#     df_num.iloc[i].fillna(value, inplace=True)
# #df = pd.concat([df_num, df_cat])
# df_num.head()

'The below approach of converting categorical cols and numerical cols separately does not work as it leads to complications in the arrangement of columns and hence we avoid this way...'

In [65]:
"""Imputation"""
for i in cc_apps.columns:
    if cc_apps[i].dtype == "object":
        value = cc_apps[i].value_counts()[0] # getting the most frequent value
        cc_apps.iloc[i].replace(np.nan, value, inplace=True)
    else:
        value = cc_apps[i].mean() # getting the most frequent value
        cc_apps.iloc[i].fillna(value, inplace=True)
        

In [66]:
## One-Hot Encoding
cc_apps = pd.get_dummies(cc_apps, drop_first=True)
cc_apps.head(7)

,2,7,10,12,0_b,1_15.17,1_15.75,1_15.83,1_15.92,1_16.00,1_16.08,1_16.17,1_16.25,1_16.33,1_16.50,1_16.92,1_17.08,1_17.25,1_17.33,1_17.42,1_17.50,1_17.58,1_17.67,1_17.83,1_17.92,1_18.00,1_18.08,1_18.17,1_18.25,1_18.33,1_18.42,1_18.50,1_18.58,1_18.67,1_18.75,1_18.83,1_18.92,1_19.00,1_19.17,1_19.33,...,1_65.42,1_67.75,1_68.67,1_69.17,1_69.50,1_71.58,1_73.42,1_74.83,1_76.75,1_80.25,3_u,3_y,4_gg,4_p,5_c,5_cc,5_d,5_e,5_ff,5_i,5_j,5_k,5_m,5_q,5_r,5_w,5_x,6_dd,6_ff,6_h,6_j,6_n,6_o,6_v,6_z,8_t,9_t,11_p,11_s,13_-
0,0.000,1.25,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0
1,4.460,3.04,6,560,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0
2,0.500,1.50,0,824,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
3,1.540,3.75,5,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0
4,5.625,1.71,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0
5,4.000,2.50,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
6,1.040,6.50,0,31285,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0


In [67]:
# Preparation
## Splitting the datasets
from sklearn.model_selection import train_test_split

X = cc_apps.iloc[:,:-1].values  # features
y = cc_apps.iloc[:,-1].values  # target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [68]:
# Scaling 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)  # forgot to scale the test chunk

In [69]:
# Training
## Model selection
model = LogisticRegression()
model.fit(X_train, y_train)
train_pred = model.predict(X_train) # first test on the training data chunk
#print(f"Prediction: {pred}")
best_score_test = model.score(X_train, y_train)
best_score_test

0.9909420289855072

In [70]:
# Plotting
print(confusion_matrix(y_train, train_pred))

[[235   2]
 [  3 312]]


In [71]:
"""Fine Tunning"""
# THe hyperparameters : C, mac_iter, penalty
param_grid = {'C': [0.01, 0.1, 1], 'max_iter': [100, 10, 1000]}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_score = grid_search.best_score_
best_score

0.8477641277641277

In [72]:
print(confusion_matrix(y_train, train_pred))

[[235   2]
 [  3 312]]
